In [1]:
import pandas as pd
import dask.dataframe as ddf

In [86]:
c_1 = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/its-소통/1/its_c_1_1_m1.csv', dtype='object')
e_1 = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/its-돌발/1/its_e_1_1_m1.csv', dtype='object')
w_1 = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/기상청-방재/weather_1_m1.csv')
r = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/도로 데이터/road(info)_m1.csv')
area = ddf.read_csv(
    '/Volumes/Expansion/traffic-prediction/data/전국교통사고다발지역표준데이터/전국교통사고다발지역표준데이터.csv')
cam = ddf.read_csv(
    '/Volumes/Expansion/traffic-prediction/data/전국무인교통단속카메라표준데이터/전국무인교통단속카메라표준데이터_m1.csv')
node = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/표준노드링크/data/node_m1.csv')
link = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/표준노드링크/data/link_m1.csv')
men = ddf.read_csv(
    '/Volumes/Expansion/traffic-prediction/data/행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수/행정안전부_지역별(행정동) 성별 연령별 주민등록 인구수_20240731.csv')
c = ddf.read_csv('/Volumes/Expansion/traffic-prediction/data/혼잡빈도/confusion_1_m1.csv')

In [63]:
c1h = c_1.tail()
c1h

,생성일,생성시분,링크ID,통행속도,통행시간
2025573,20230101,2355,4180383901,4.0,0
2025574,20230101,2355,4180384001,97.0,0
2025575,20230101,2355,4180384101,76.0,0
2025576,20230101,2355,4180384201,115.0,0
2025577,20230101,2355,4180384301,79.0,0


In [64]:
def get_big_date(v):
    a = str(v)
    return a[:4] + '-' + a[4:6] + '-' + a[6:8]

In [65]:
def get_time(v):
    a = str(v).ljust(4, '0')
    return a[:2] + ':' + a[2:]

In [66]:
get_time(0)

'00:00'

In [67]:
c1h['date'] = pd.to_datetime(c1h['생성일'].apply(get_big_date) + ' ' + c1h['생성시분'].apply(get_time))

In [68]:
c1h

,생성일,생성시분,링크ID,통행속도,통행시간,date
2025573,20230101,2355,4180383901,4.0,0,2023-01-01 23:55:00
2025574,20230101,2355,4180384001,97.0,0,2023-01-01 23:55:00
2025575,20230101,2355,4180384101,76.0,0,2023-01-01 23:55:00
2025576,20230101,2355,4180384201,115.0,0,2023-01-01 23:55:00
2025577,20230101,2355,4180384301,79.0,0,2023-01-01 23:55:00


In [81]:
pd.Index([c1h.columns[-1]]).append(c1h.columns[:-1])

Index(['date', '생성일', '생성시분', '링크ID', '통행속도', '통행시간'], dtype='object')

In [83]:
c1h = c1h[pd.Index([c1h.columns[-1]]).append(c1h.columns[:-1])]

In [84]:
c1h

,date,생성일,생성시분,링크ID,통행속도,통행시간
2025573,2023-01-01 23:55:00,20230101,2355,4180383901,4.0,0
2025574,2023-01-01 23:55:00,20230101,2355,4180384001,97.0,0
2025575,2023-01-01 23:55:00,20230101,2355,4180384101,76.0,0
2025576,2023-01-01 23:55:00,20230101,2355,4180384201,115.0,0
2025577,2023-01-01 23:55:00,20230101,2355,4180384301,79.0,0


In [151]:
e1h = e_1.compute()

In [152]:
e1h

,돌발일시,관리기관,도로타입,도로번호,도로명,방향,돌발구분,돌발상세구분,돌발내용,X,Y,링크아이디,돌발종료일시
0,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,128.4929,37.9769,2650168000,-
1,2023-01-01 12:41:17,한국도로공사,고속국도,45,중부내륙선,창원 방향,기타돌발,차량증가/정체,차량증가/정체,127.6062,37.2134,2370601900,-
2,2023-01-01 11:05:13,한국도로공사,고속국도,65,동해선(부산-포항),포항 방향,교통사고,사고,(갓길)승용차관련사고처리중,129.4162,35.7734,3520264800,-
3,2023-01-01 15:48:21,한국도로공사,고속국도,40,평택제천선,제천 방향,기타돌발,차량증가/정체,차량증가/정체,127.6166,36.9861,2810051300,-
4,2023-01-01 14:13:00,한국도로공사,고속국도,25,논산천안선,천안 방향,기타돌발,차량증가/정체,-차령터널(263K)3km정체,127.1026,36.0819,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,2023-01-01 15:04:00,경찰청,시군도,0,가양대로,-,공사,-,<공사> 가양로 구룡사거리 마포구고양시덕양구 1차로 주의운전 도로공사 -,126.872128,37.581869,2180147300,2023-01-01 18:00:00
1119,2023-01-01 15:12:00,경찰청,일반국도,38,서동대로,-,공사,-,<공사> 일반국도38호선 안양교앞교차로 내리사거리 2차로 주의운전 도로공사 -,127.215717,37.009206,2310014402,2023-01-01 17:00:00
1120,2023-01-01 15:22:00,경찰청,특별광역시도,0,계백로1716번길,-,공사,-,<공사> 계백로1716번길 센트럴파크3차 글꽃초등학교사거리 1차로 주의운전 도로공사 -,127.411943,36.319642,1840126300,2023-01-01 18:00:00
1121,2023-01-01 16:19:00,경찰청,특별광역시도,0,국회대로,-,교통사고,-,<사고> 국회대로 여의2교남단교차로 의원회관앞교차로 4차로 주의운전 교통사고 서울청,126.912093,37.526136,1180031502,2023-01-01 16:30:00


In [153]:
def remove_s(v):
    a = str(v)
    return a[:-2] + '00'

In [154]:
remove_s('2023-01-01 15:48:21')

'2023-01-01 15:48:00'

In [155]:
e1h['date'] = pd.to_datetime(e1h['돌발일시'].apply(remove_s))

In [156]:
e1h

,돌발일시,관리기관,도로타입,도로번호,도로명,방향,돌발구분,돌발상세구분,돌발내용,X,Y,링크아이디,돌발종료일시,date
0,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,128.4929,37.9769,2650168000,-,2023-01-01 13:12:00
1,2023-01-01 12:41:17,한국도로공사,고속국도,45,중부내륙선,창원 방향,기타돌발,차량증가/정체,차량증가/정체,127.6062,37.2134,2370601900,-,2023-01-01 12:41:00
2,2023-01-01 11:05:13,한국도로공사,고속국도,65,동해선(부산-포항),포항 방향,교통사고,사고,(갓길)승용차관련사고처리중,129.4162,35.7734,3520264800,-,2023-01-01 11:05:00
3,2023-01-01 15:48:21,한국도로공사,고속국도,40,평택제천선,제천 방향,기타돌발,차량증가/정체,차량증가/정체,127.6166,36.9861,2810051300,-,2023-01-01 15:48:00
4,2023-01-01 14:13:00,한국도로공사,고속국도,25,논산천안선,천안 방향,기타돌발,차량증가/정체,-차령터널(263K)3km정체,127.1026,36.0819,-,-,2023-01-01 14:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,2023-01-01 15:04:00,경찰청,시군도,0,가양대로,-,공사,-,<공사> 가양로 구룡사거리 마포구고양시덕양구 1차로 주의운전 도로공사 -,126.872128,37.581869,2180147300,2023-01-01 18:00:00,2023-01-01 15:04:00
1119,2023-01-01 15:12:00,경찰청,일반국도,38,서동대로,-,공사,-,<공사> 일반국도38호선 안양교앞교차로 내리사거리 2차로 주의운전 도로공사 -,127.215717,37.009206,2310014402,2023-01-01 17:00:00,2023-01-01 15:12:00
1120,2023-01-01 15:22:00,경찰청,특별광역시도,0,계백로1716번길,-,공사,-,<공사> 계백로1716번길 센트럴파크3차 글꽃초등학교사거리 1차로 주의운전 도로공사 -,127.411943,36.319642,1840126300,2023-01-01 18:00:00,2023-01-01 15:22:00
1121,2023-01-01 16:19:00,경찰청,특별광역시도,0,국회대로,-,교통사고,-,<사고> 국회대로 여의2교남단교차로 의원회관앞교차로 4차로 주의운전 교통사고 서울청,126.912093,37.526136,1180031502,2023-01-01 16:30:00,2023-01-01 16:19:00


In [157]:
def move_date_first(a):
    return pd.Index([a.columns[-1]]).append(a.columns[:-1])

In [158]:
e1h = e1h[move_date_first(e1h)]

In [159]:
e1h

,date,돌발일시,관리기관,도로타입,도로번호,도로명,방향,돌발구분,돌발상세구분,돌발내용,X,Y,링크아이디,돌발종료일시
0,2023-01-01 13:12:00,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,128.4929,37.9769,2650168000,-
1,2023-01-01 12:41:00,2023-01-01 12:41:17,한국도로공사,고속국도,45,중부내륙선,창원 방향,기타돌발,차량증가/정체,차량증가/정체,127.6062,37.2134,2370601900,-
2,2023-01-01 11:05:00,2023-01-01 11:05:13,한국도로공사,고속국도,65,동해선(부산-포항),포항 방향,교통사고,사고,(갓길)승용차관련사고처리중,129.4162,35.7734,3520264800,-
3,2023-01-01 15:48:00,2023-01-01 15:48:21,한국도로공사,고속국도,40,평택제천선,제천 방향,기타돌발,차량증가/정체,차량증가/정체,127.6166,36.9861,2810051300,-
4,2023-01-01 14:13:00,2023-01-01 14:13:00,한국도로공사,고속국도,25,논산천안선,천안 방향,기타돌발,차량증가/정체,-차령터널(263K)3km정체,127.1026,36.0819,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,2023-01-01 15:04:00,2023-01-01 15:04:00,경찰청,시군도,0,가양대로,-,공사,-,<공사> 가양로 구룡사거리 마포구고양시덕양구 1차로 주의운전 도로공사 -,126.872128,37.581869,2180147300,2023-01-01 18:00:00
1119,2023-01-01 15:12:00,2023-01-01 15:12:00,경찰청,일반국도,38,서동대로,-,공사,-,<공사> 일반국도38호선 안양교앞교차로 내리사거리 2차로 주의운전 도로공사 -,127.215717,37.009206,2310014402,2023-01-01 17:00:00
1120,2023-01-01 15:22:00,2023-01-01 15:22:00,경찰청,특별광역시도,0,계백로1716번길,-,공사,-,<공사> 계백로1716번길 센트럴파크3차 글꽃초등학교사거리 1차로 주의운전 도로공사 -,127.411943,36.319642,1840126300,2023-01-01 18:00:00
1121,2023-01-01 16:19:00,2023-01-01 16:19:00,경찰청,특별광역시도,0,국회대로,-,교통사고,-,<사고> 국회대로 여의2교남단교차로 의원회관앞교차로 4차로 주의운전 교통사고 서울청,126.912093,37.526136,1180031502,2023-01-01 16:30:00


In [160]:
lh= link.compute()

In [161]:
lh

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,ROAD_NO,ROAD_NAME,ROAD_USE,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,C-ITS,LENGTH,UPDATEDATE,geometry
0,2710002605,2710246000,2710245800,2,101,0,35,ÁßºÎ°í¼Óµµ·Î,0,0,0,110,5,0,0,0,802.513275,20240223,LINESTRING (237463.82864681576 448494.14197941...
1,2710002506,2710246300,2710008400,2,101,0,35,ÁßºÎ°í¼Óµµ·Î,0,0,0,110,5,0,0,0,911.416276,20240223,LINESTRING (237391.9446491873 449402.386403182...
2,2710002607,2710246200,2710246000,2,101,0,35,ÁßºÎ°í¼Óµµ·Î,0,0,0,110,5,0,0,0,918.955738,20240223,LINESTRING (237384.37150444122 449403.15866465...
3,2710706900,2710245000,2710244600,2,107,0,-,-,0,0,0,40,0,0,0,0,59.268985,20240223,LINESTRING (237671.09146364938 446586.04185184...
4,2150268601,2150102800,2150102900,1,107,0,-,-,0,0,0,40,0,0,0,0,7.638523,20231211,LINESTRING (205113.4722244901 596106.180547790...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73676,2870019411,2870008101,2870078400,2,101,0,15,¼­ÇØ¾È°í¼Óµµ·Î,0,0,0,110,5,0,0,0,11066.335812,20230519,LINESTRING (160193.08210000023 419974.33589999...
73677,3130000103,3130000101,3130000106,1,106,0,742,¹éÀå·Î,0,0,0,60,0,0,0,0,5620.608180,20230519,LINESTRING (240299.19749999978 338592.37050000...
73678,3510029708,3510012607,3510015403,1,105,0,20,ÇØ¾È·Î,0,0,0,50,0,0,0,0,2814.570321,20230519,LINESTRING (415735.5536000002 398641.552600000...
73679,1860014904,1860006902,1860007000,1,104,0,-,±Ý³²±¸Áï·Î,0,0,0,30,0,0,0,0,1708.927407,20230519,LINESTRING (234456.0343000004 427806.905200000...


In [162]:
lh.rename(columns={'ROAD_NO':'도로번호'}, inplace=True)

In [163]:
lh

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,도로번호,ROAD_NAME,ROAD_USE,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,C-ITS,LENGTH,UPDATEDATE,geometry
0,2710002605,2710246000,2710245800,2,101,0,35,ÁßºÎ°í¼Óµµ·Î,0,0,0,110,5,0,0,0,802.513275,20240223,LINESTRING (237463.82864681576 448494.14197941...
1,2710002506,2710246300,2710008400,2,101,0,35,ÁßºÎ°í¼Óµµ·Î,0,0,0,110,5,0,0,0,911.416276,20240223,LINESTRING (237391.9446491873 449402.386403182...
2,2710002607,2710246200,2710246000,2,101,0,35,ÁßºÎ°í¼Óµµ·Î,0,0,0,110,5,0,0,0,918.955738,20240223,LINESTRING (237384.37150444122 449403.15866465...
3,2710706900,2710245000,2710244600,2,107,0,-,-,0,0,0,40,0,0,0,0,59.268985,20240223,LINESTRING (237671.09146364938 446586.04185184...
4,2150268601,2150102800,2150102900,1,107,0,-,-,0,0,0,40,0,0,0,0,7.638523,20231211,LINESTRING (205113.4722244901 596106.180547790...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73676,2870019411,2870008101,2870078400,2,101,0,15,¼­ÇØ¾È°í¼Óµµ·Î,0,0,0,110,5,0,0,0,11066.335812,20230519,LINESTRING (160193.08210000023 419974.33589999...
73677,3130000103,3130000101,3130000106,1,106,0,742,¹éÀå·Î,0,0,0,60,0,0,0,0,5620.608180,20230519,LINESTRING (240299.19749999978 338592.37050000...
73678,3510029708,3510012607,3510015403,1,105,0,20,ÇØ¾È·Î,0,0,0,50,0,0,0,0,2814.570321,20230519,LINESTRING (415735.5536000002 398641.552600000...
73679,1860014904,1860006902,1860007000,1,104,0,-,±Ý³²±¸Áï·Î,0,0,0,30,0,0,0,0,1708.927407,20230519,LINESTRING (234456.0343000004 427806.905200000...


In [164]:
e1h

,date,돌발일시,관리기관,도로타입,도로번호,도로명,방향,돌발구분,돌발상세구분,돌발내용,X,Y,링크아이디,돌발종료일시
0,2023-01-01 13:12:00,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,128.4929,37.9769,2650168000,-
1,2023-01-01 12:41:00,2023-01-01 12:41:17,한국도로공사,고속국도,45,중부내륙선,창원 방향,기타돌발,차량증가/정체,차량증가/정체,127.6062,37.2134,2370601900,-
2,2023-01-01 11:05:00,2023-01-01 11:05:13,한국도로공사,고속국도,65,동해선(부산-포항),포항 방향,교통사고,사고,(갓길)승용차관련사고처리중,129.4162,35.7734,3520264800,-
3,2023-01-01 15:48:00,2023-01-01 15:48:21,한국도로공사,고속국도,40,평택제천선,제천 방향,기타돌발,차량증가/정체,차량증가/정체,127.6166,36.9861,2810051300,-
4,2023-01-01 14:13:00,2023-01-01 14:13:00,한국도로공사,고속국도,25,논산천안선,천안 방향,기타돌발,차량증가/정체,-차령터널(263K)3km정체,127.1026,36.0819,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118,2023-01-01 15:04:00,2023-01-01 15:04:00,경찰청,시군도,0,가양대로,-,공사,-,<공사> 가양로 구룡사거리 마포구고양시덕양구 1차로 주의운전 도로공사 -,126.872128,37.581869,2180147300,2023-01-01 18:00:00
1119,2023-01-01 15:12:00,2023-01-01 15:12:00,경찰청,일반국도,38,서동대로,-,공사,-,<공사> 일반국도38호선 안양교앞교차로 내리사거리 2차로 주의운전 도로공사 -,127.215717,37.009206,2310014402,2023-01-01 17:00:00
1120,2023-01-01 15:22:00,2023-01-01 15:22:00,경찰청,특별광역시도,0,계백로1716번길,-,공사,-,<공사> 계백로1716번길 센트럴파크3차 글꽃초등학교사거리 1차로 주의운전 도로공사 -,127.411943,36.319642,1840126300,2023-01-01 18:00:00
1121,2023-01-01 16:19:00,2023-01-01 16:19:00,경찰청,특별광역시도,0,국회대로,-,교통사고,-,<사고> 국회대로 여의2교남단교차로 의원회관앞교차로 4차로 주의운전 교통사고 서울청,126.912093,37.526136,1180031502,2023-01-01 16:30:00


In [165]:
e1h_lh = pd.merge(e1h, lh, on='도로번호', how='inner')

In [166]:
e1h_lh

,date,돌발일시,관리기관,도로타입,도로번호,도로명,방향,돌발구분,돌발상세구분,돌발내용,...,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,C-ITS,LENGTH,UPDATEDATE,geometry
0,2023-01-01 13:12:00,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,...,0,0,60,0,0,0,0,302.460731,20230519,LINESTRING (346143.95770000014 306810.93329999...
1,2023-01-01 13:12:00,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,...,0,0,100,5,0,0,0,191.169306,20231211,LINESTRING (228780.90549726193 559319.03821418...
2,2023-01-01 13:12:00,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,...,0,1,30,5,0,0,0,331.293348,20230519,LINESTRING (268854.74980000034 573434.00860000...
3,2023-01-01 13:12:00,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,...,0,0,100,5,0,0,0,1487.113039,20230519,LINESTRING (280412.2002999997 572631.064400000...
4,2023-01-01 13:12:00,2023-01-01 13:12:27,한국도로공사,고속국도,60,서울양양선,양양 방향,기타돌발,고장,(갓길)승용차고장처리중,...,0,0,100,5,0,0,0,749.264124,20230519,LINESTRING (231604.18350000028 559447.49159999...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095099,2023-01-01 18:56:00,2023-01-01 18:56:00,경찰청,고속국도,30,당진영덕고속도로,-,교통사고,-,<사고> 청주영덕고속도로 안동시의성군 안동JC 2차로 156 Km (안동JC 방향)...,...,0,0,60,0,0,0,0,330.566830,20230519,LINESTRING (347144.24509999994 356580.17439999...
3095100,2023-01-01 18:56:00,2023-01-01 18:56:00,경찰청,고속국도,30,당진영덕고속도로,-,교통사고,-,<사고> 청주영덕고속도로 안동시의성군 안동JC 2차로 156 Km (안동JC 방향)...,...,0,0,110,5,0,0,0,897.379291,20230519,LINESTRING (251322.3245000001 434843.213899999...
3095101,2023-01-01 18:56:00,2023-01-01 18:56:00,경찰청,고속국도,30,당진영덕고속도로,-,교통사고,-,<사고> 청주영덕고속도로 안동시의성군 안동JC 2차로 156 Km (안동JC 방향)...,...,0,0,60,0,0,0,0,1659.499197,20230519,LINESTRING (320818.9740000004 286548.440500000...
3095102,2023-01-01 18:56:00,2023-01-01 18:56:00,경찰청,고속국도,30,당진영덕고속도로,-,교통사고,-,<사고> 청주영덕고속도로 안동시의성군 안동JC 2차로 156 Km (안동JC 방향)...,...,0,0,80,0,0,0,0,363.114206,20230519,"LINESTRING (170859.2572999997 348338.3794, 170..."
